In [24]:
import tensorflow as tf
import core.utils as utils
from tensorflow.python.saved_model import tag_constants
import cv2
import numpy as np

In [25]:
# set parameters
MODEL_PATH = './checkpoints/yolov4-416'
IOU_THRESHOLD = 0.45
SCORE_THRESHOLD = 0.25
INPUT_SIZE = 416

In [30]:
# load model
saved_model_loaded = tf.saved_model.load(MODEL_PATH, tags=[tag_constants.SERVING])
yolo_v4 = saved_model_loaded.signatures['serving_default']

In [27]:
def main(img_path):
    #이미지를 로드합니다.
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #이미지를 전처리합니다.
    img_input = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
    img_input = img_input / 255.
    img_input = img_input[np.newaxis, ...].astype(np.float32)
    img_input = tf.constant(img_input)

    #Yolo 모델을 이용해 입력 이미지에서 예측 박스를 받습니다. 
    pred_bbox = yolo_v4(img_input)

    for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]

    boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
        boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape(
            pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
        max_output_size_per_class=50,
        max_total_size=50,
        iou_threshold=IOU_THRESHOLD,
        score_threshold=SCORE_THRESHOLD
    )

    pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]

    #예측된 박스를 원본 이미지에 그려줍니다.
    #car만 출력하기 위해 utils.draw_bbox를 수정했습니다.
    result = utils.draw_bbox(img, pred_bbox)

    result = cv2.cvtColor(np.array(result), cv2.COLOR_RGB2BGR)

    save_path = img_path.split("/")
    img_name = save_path.pop()
    save_path = "/".join(save_path)+"/result_"+img_name

    cv2.imwrite(save_path, result)
    cv2.imshow(save_path, result)


In [31]:
img_path = './data/road_kor.jpg'
main(img_path)
print("Done!!")

Done!!
